In [ ]:
import cv2
import numpy as np
import pylab as pl

# 读取两张图片
image1 = cv2.imread('left.jpg')
image2 = cv2.imread('right.jpg')

# 转换为灰度图像
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# 使用SIFT特征检测器和描述符提取关键点和特征向量
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

# 使用FLANN匹配器进行特征匹配
flann = cv2.FlannBasedMatcher()
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# 根据最近邻比率进行筛选
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 提取匹配的关键点坐标
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# 使用RANSAC算法估计单应性矩阵
H, _ = cv2.findHomography(src_pts, dst_pts,cv2.RANSAC,5.0)

# 将image1投影到image2上
result = cv2.warpPerspective(image1, H, (image2.shape[1], image2.shape[0]))

# 显示结果
pl.figure(), pl.imshow(result[:, :, ::-1]), pl.title('out'), pl.show()

cv2.imwrite("result.jpg",result)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("单应性矩阵:")
print(H)